In [1]:
import os
import numpy as np
from dataclasses import dataclass, fields
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

from pyiron.atomistics.structure.atoms import ase_to_pyiron
from pyiron.atomistics.structure.atoms import pyiron_to_ase

In [2]:
magnetic_info = []
magnetic_info.append([[0, 0, 1], [0, 0, 2], [0, 0, 3], [0, 0, 4], [0, 0, 5]])
magnetic_info.append([0.0, 1.0, 1.0, 1.0, 0.0])
magnetic_info.append([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])

In [3]:
a = 3.95
alpha = 90
structure = crystal(('Ga', 'Mn', 'N'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                       spacegroup=221,
                       cellpar=[a, a, a, alpha, alpha, alpha])
structure.symbols.species()

{'Ga', 'Mn', 'N'}

In [4]:
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

vasp.io.constr_INCAR_parameters.I_CONSTRAINED = "4"
vasp.structure.kpoints = "2 2 2"
vasp.io.df = [structure, magnetic_info]
vasp.io.df

elements              positions    magmoms  betahs  B_CONSTRs
0       Ga        [0.0, 0.0, 0.0]  [0, 0, 1]     0.0  [0, 0, 0]
1       Mn    [0.0, 1.975, 1.975]  [0, 0, 2]     1.0  [0, 0, 0]
2       Mn    [1.975, 0.0, 1.975]  [0, 0, 3]     1.0  [0, 0, 0]
3       Mn    [1.975, 1.975, 0.0]  [0, 0, 4]     1.0  [0, 0, 0]
4        N  [1.975, 1.975, 1.975]  [0, 0, 5]     0.0  [0, 0, 0]

In [5]:
vasp.prepare_calculation()

{'Ga', 'Mn', 'N'}


In [13]:
vasp.structure.species

['Ga', 'Ni', 'Mn']

In [6]:
vasp.magnetism.betahs

[0.0, 1.0, 1.0, 1.0, 0.0]

In [8]:
structure.set_initial_magnetic_moments([[0, 0, 2], [0, 0, 0], [0, 0, 2], [0, 0, 2], [0, 0, 2]])

ValueError: magmons can be collinear or non-collinear.

In [9]:
structure = ase_to_pyiron(structure)
Nrepeat=2
structure.set_repeat([Nrepeat,Nrepeat,Nrepeat])

In [ ]:
ase.build.make_supercell(structure, )

In [26]:
structure.get_initial_magnetic_moments()

array([[0., 0., 2.],
       [0., 0., 0.],
       [0., 0., 2.],
       [0., 0., 2.],
       [0., 0., 2.]])

In [3]:
list( structure.symbols.species() )

['Mn', 'Ni', 'Ga']

In [4]:
structure.get_chemical_symbols()

['Ga', 'Mn', 'Mn', 'Mn', 'Ni']

In [5]:
structure.get_chemical_symbols().count("Mn")

3

In [6]:
structure.cell.array

array([[3.95, 0.  , 0.  ],
       [0.  , 3.95, 0.  ],
       [0.  , 0.  , 3.95]])

In [7]:
list( structure.positions )

[array([0., 0., 0.]),
 array([0.   , 1.975, 1.975]),
 array([1.975, 0.   , 1.975]),
 array([1.975, 1.975, 0.   ]),
 array([1.975, 1.975, 1.975])]

In [8]:
structure.positions

array([[0.   , 0.   , 0.   ],
       [0.   , 1.975, 1.975],
       [1.975, 0.   , 1.975],
       [1.975, 1.975, 0.   ],
       [1.975, 1.975, 1.975]])

Inputs

- positions
- magmoms
- element
- betah

In [9]:
lattice = pd.DataFrame({
   "avec"  : structure.cell[0],
   "bvec"  : structure.cell[1],
   "cvec"  : structure.cell[2]
})
lattice

avec  bvec  cvec
0  3.95  0.00  0.00
1  0.00  3.95  0.00
2  0.00  0.00  3.95

In [28]:
atoms = pd.DataFrame({
   "elements"  : structure.get_chemical_symbols(),
   "positions" : list( structure.positions ),
   "magmoms"   : list( structure.get_initial_magnetic_moments() ),
   "betahs"    : [1.0, False, 1.0, 0.3, 0.2],
   "B_CONSTRs" : [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]
})
atoms = atoms.sort_values("elements")
atoms

elements              positions          magmoms betahs  B_CONSTRs
0       Ga        [0.0, 0.0, 0.0]  [0.0, 0.0, 2.0]    1.0  [0, 0, 0]
1       Mn    [0.0, 1.975, 1.975]  [0.0, 0.0, 0.0]  False  [0, 0, 0]
2       Mn    [1.975, 0.0, 1.975]  [0.0, 0.0, 2.0]    1.0  [0, 0, 0]
3       Mn    [1.975, 1.975, 0.0]  [0.0, 0.0, 2.0]    0.3  [0, 0, 0]
4       Ni  [1.975, 1.975, 1.975]  [0.0, 0.0, 2.0]    0.2  [0, 0, 1]

In [11]:
elements = atoms["elements"].tolist()
elements_red = list(dict.fromkeys(elements))
print(elements)
print(elements_red)

['Ga', 'Mn', 'Mn', 'Mn', 'Ni']
['Ga', 'Mn', 'Ni']


In [12]:
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

vasp.io.constr_INCAR_parameters.I_CONSTRAINED = "5"
vasp.structure.kpoints = "2 2 2"
vasp.prepare_calculation(atoms, structure)

In [13]:
vasp.io.cwd = '/home/emendive/workbench/devel/gitclones/VASP_job/'
vasp.io.cwd

'/home/emendive/workbench/devel/gitclones/VASP_job/'

In [14]:
vasp.io.constr_INCAR_parameters.I_CONSTRAINED = "5"
vasp.structure.kpoints = "2 2 2"
vasp.prepare_calculation(atoms, structure)

In [15]:
vasp.structure.elements

['Ga', 'Mn', 'Mn', 'Mn', 'Ni']

In [16]:
vasp.structure.species

['Mn', 'Ni', 'Ga']

In [9]:
vasp.structure.elements.count("Mn")

2

In [13]:
vasp.io.constr_INCAR_parameters.I_CONSTRAINED

'5'

In [14]:
vasp.magnetism.B_CONSTRs

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [17]:
vasp.io.cwd

'/home/emendive/workbench/devel/gitclones/VASP_job/test/'

In [18]:
vasp.io.standard_INCAR_parameters

standard_INCAR_parameters(SYSTEM='my material #jobname', PREC='Accurate', ALGO='Fast', NPAR='8', LREAL='.FALSE.', LWAVE='.FALSE.', LCHARG='.TRUE.', LORBIT='10', ISMEAR='1', SIGMA='0.03', ISTART='0', ICHARG='2', ISPIN='2', ENCUT='500', EDIFF='1e-6', NELM='200', LMAXMIX='6', LNONCOLLINEAR='.TRUE.')

In [19]:
vasp.structure.kpoints = "1 1 1"
vasp.structure.kpoints

'1 1 1'

In [21]:
vasp.structure.elements = ["Ni"]
vasp.io.potential_files.Ni = "Ni"

In [6]:
class A(object):
    def __init__(self, val):
        self.idx = val

    @property    
    def idx(self):
        return self._idx

    @idx.setter
    def idx(self, val):
        try:
            value1, value2 = val
        except ValueError:
            raise ValueError("Pass an iterable with two items")
        else:
            """ This will run only if no exception was raised """
            self._idx = value1+value2

In [7]:
my_A = A([1,2])
my_A.idx
my_A.idx = [1,3]
my_A.idx

4